# WindFarmer API demo
The WindFarmer API is a web API. You can call it from any coding language, or use tools like Postman. There is an OpenAPI definition which provides
documentation, and allows client code to be generated.

From python you can call the API directly, using `urllib3` or `requests`.

## Using the API directly
First, import the necessary modules

In [1]:
import os
import requests
import json
import time

To access the API you need a authorization token. This should be kept secure - and not added to source control, so I'm getting it from an environment variable.

The token should be passed as an Authorization header. We also need to set the `Content-Type` to let the API know that we're sending JSON data.

In [2]:
api_url = 'https://windfarmer.dnv.com/api/v1/'
auth_token = auth_token = os.environ['WINDFARMER_ACCESS_KEY']

headers = {
    'Authorization': f'Bearer {auth_token}',
    'Content-Type': 'application/json'
}

### Call `Status`
Try calling the `Status` endpoint. This verifies that you can access the API and that your token is valid.

In [3]:
response = requests.get(api_url + 'Status', headers = headers)
print(f'Response from Status: {response.status_code}')
print(response.text)

Response from Status: 200
{"message":"Connection to DNV WindFarmer Services API was successful.","windFarmerServicesAPIVersion":"1.0.28","calculationLibraryVersion":"1.1.13.0"}


### Call `BlockageCorrection`

Prepare the input data for the BlockageCorrection calculation. This could come from elsewhere in your script, but here we're just loading it from a file.

In [4]:
with open('input/BlockageCorrection_input_data.json') as f:
    json_string = f.read()
    input_data = json.loads(json_string)

print(json_string[0:500] +  '...')


{
    "turbines": [
       { "easting_m": 199977, "northing_m": 2242510, "hubHeight_m": 100, "rotorDiameter_m": 80 },
       { "easting_m": 200556, "northing_m": 2242405, "hubHeight_m": 100, "rotorDiameter_m": 80 },
       { "easting_m": 199398, "northing_m": 2242615, "hubHeight_m": 100, "rotorDiameter_m": 80 },
       { "easting_m": 199548, "northing_m": 2242300, "hubHeight_m": 100, "rotorDiameter_m": 80 },
       { "easting_m": 200127, "northing_m": 2242195, "hubHeight_m": 100, "rotorDiameter_...


The data is now in a dictionary...

In [5]:
input_data['turbines'][1]['easting_m']

200556

Send the input data to the Blockage Correction calculation

In [6]:
start = time.time()
response = requests.post(
    api_url + 'BlockageCorrection', 
    headers=headers,
    json = input_data)
    
print(f'Response {response.status_code} in {time.time() - start:.2f}s')

Response 200 in 1.20s


Deserialize the response, and use it...

In [7]:
result = json.loads(response.content)
blockage_correction = float(result['blockageEffect'])* 100
print(f"blockage correction = {blockage_correction:.3f} % ")

blockage correction = 98.835 % 
